In [ ]:
# this is a little trick to make sure the the notebook takes up most of the screen:
from IPython.display import HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# Recommendation to leave the logging config like this, otherwise you'll be flooded with unnecessary info
import logging
logging.basicConfig(level=logging.WARNING, format='%(levelname)s:%(message)s')  

# README

This notebook uses the the models defined in the [Syntax and Semantics introduction](./Syntax-Semantics.ipynb). If you're unfamiliar with CREST we suggest you start there.

If you are unfamiliar with Jupyter notebooks, please consult appropriate tutorials and introductions, e.g. [here](https://jupyter.readthedocs.io/en/latest/content-quickstart.html).

**This notebook** was designed to execute all cells from top to bottom. You can either iteratively execute each cell or simply execute all at once:  
Menu `Cell` -> `Run all`  

<p><b><font size="3"> Importing </font></b></p>

The following lines do the following:   
`%%capture` suppresses the output of the current cell (otheriw.  
`%run filename` executes a notebook in the current context. We use it to import things from other notebooks.

*Note, that this is a very dumb version of importing. It executes everything (i.e. it's slow) and also localises all variables (i.e. susceptible to variable shadowing). We're waiting for smarter functionality (hopefully in the next IPython version).*

In [ ]:
%%capture
%run Syntax-Semantics.ipynb

The simulator is provided in a Python library, so we can import it using the standard Python functionality

In [ ]:
# import the simulator
from crestdsl.simulation import Simulator

# Using the Simulator
A simulator is initialized with a root-entity and a time domain.
For convenience we can plot directly from the simulator.

The plotting should present the entity in its initial state. It is plotted exactly as if using the `elk` plotter directly, except that it shows the current time, which is set to 0 (check the root-entity's label)


In [ ]:
gl = GrowLamp()
sim = Simulator(gl)
sim.plot()

## Stabilisation
The simulator will execute the system until it is stable. 
That is, until there is a point where no more transitions can be triggered and all updates/influences/actions have been executed. 
In our example look specifically at the **output** values, they are now correct (instead of the dummy default values)!!

*Note: the value of room_temp_in is 21.999999...  This is a result of the calculation: (71.6-32)*5/9*.  
It has to do with Python's floating point calculations. 
The next version of CREST will use SymPy to perform calculations symbolically and have precise results.*

In [ ]:
sim.stabilise()
sim.plot()

## Modification of input values

When we modify input port values, we see that nothing happens except what we explicitly changed.  

In [ ]:
# modify the growlamp instance's inputs directly, the simulator points to that object and will use it
gl.electricity.value = 500
gl.switch.value = "on"
sim.plot()

It actually takes a *stabilise* step to propagate the values

In [ ]:
sim.stabilise()
sim.plot()

## Time advance
The *advance(dt)* function can forward time.  
Below we advance 500 time steps. The effect is that the time is now (t=500) and that on_time has the value of 500 too!

In [ ]:
sim.advance(500)
sim.plot()

## Next transition time
The simulation of time advance is based on the calculation of the next transition time.

The simulator can calculate this time by analysing system's updates and influences.  
Currently, only a limited set of features is supported in update/influence functions: variable assignments, arithmetics

*More features (conditional statements & expressions) are being developped but have to be tested first*

The example below states that the error transition will be triggered after 500.5 time steps (i.e. when on_time == 1000.5)

In [ ]:
gl = GrowLamp()
gl.electricity.value = 500
gl.switch.value = "on"

sim = Simulator(gl)
sim.stabilise()
sim.next_behaviour_change_time()

## Automatic firing of transitions
Let's assume we want to advance more than the necessary 1000.5 time steps, what happens then?
The semantics prescribe to first advance 1000.5 time steps, then fire all transitions and updates (stabilise the system), then advance the rest.

In [ ]:
gl = GrowLamp()
gl.electricity.value = 500
gl.switch.value = "on"

sim = Simulator(gl)
sim.stabilise()
sim.next_behaviour_change_time()


sim.advance(1500)  # try to modify the value here to 1000 and see that no transition happens, but if you add a value >= 1000.5 the transition is fired
sim.plot()

## Different time scales
By default the simulator will run in real-time.
However, it can be configured to use integers as time unit.
Below we have a simulator that produces two different outputs, depending on the time unit chosen.
Effectively, the system changes to the *error* state when the *on_time* reaches 1000.5 or more.

In **integer** mode the simulator will calculate that a transition happens after **1001** time units, while the **real**-valued simulator will calculate the precise point in time (**1000.5**)


In [ ]:
growlamp = GrowLamp()                 # create a growlamp instance and change inputs so it is in state *on* after stabilisation
growlamp.electricity.value = 500
growlamp.switch.value = "on"
realSimulator = Simulator(growlamp)
realSimulator.stabilise()                 # stabilise the system so it's in good state
realSimulator.timeunit = REAL
realSimulator.next_behaviour_change_time()  # should say ('to_error', 1000.5)

**Now: We will use the simulator but with a different time unit to show that the result is truly different.**

In [ ]:
growlamp = GrowLamp()                 # create a growlamp instance and change inputs so it is in state *on* after stabilisation
growlamp.electricity.value = 500
growlamp.switch.value = "on"
intSimulator = Simulator(growlamp)
intSimulator.stabilise()                 # stabilise the system so it's in good state
intSimulator.timeunit = INTEGER
intSimulator.next_behaviour_change_time()  # should say ('to_error', 1001)